# Extracting image data at possibly high resolution

In [ ]:
import siibra
from nilearn import plotting

## Intro: Spaces and Parcellations, Reference templates and maps, and their image data

`siibra` uses different layers of abstraction. On the top level, we have semantic concepts: atlases, spaces, parcellations. They are pure semantical objects, they do not contain any actual data.

In [ ]:
bigbrainspace = siibra.spaces.BIG_BRAIN
cortexparcellation = siibra.parcellations['cortex']
type(bigbrainspace),type(cortexparcellation)

The semantic objects can construct image objects. For example, a space can construct a template image, and a parcellation can construct parcellation maps in a particular space.

In [ ]:
bigbrain_template = bigbrainspace.get_template()
cortex_map = cortexparcellation.get_map(bigbrainspace)
type(bigbrain_template), type(cortex_map)

However, none of those has loaded image data yet. `siibra` follows a lazy scheme of loading image data. To get access to actual images, we apply the `fetch` method, which has an optional parameter for the physcial resolution. `fetch` returns nibabel image objects with an affine matrix pointing to the physical space.

In [ ]:
template_img = bigbrain_template.fetch(resolution_mm=0.64)
cortex_img =  cortex_map.fetch(resolution_mm=0.8)
type(template_img), type(cortex_img)

Now we can easily visualize those:

In [ ]:
def plotmap(img,tpl,opacity=1.):
    return plotting.view_img(
        img,
        bg_img=tpl,
        symmetric_cmap=False,
        resampling_interpolation='nearest',
        opacity=opacity)

In [ ]:
plotmap(cortex_img,template_img)

## Using an atlas to access spaces and parcellations

The `siibra` atlas object facilitates this process a bit, by bringing spaces and parcellations into one context.

In [ ]:
atlas = siibra.atlases['human']
atlas.select(parcellation="cortex")
template_img = atlas.get_template("bigbrain").fetch(resolution_mm=0.64)
cortex_img = atlas.get_map("bigbrain").fetch(resolution_mm=0.8)
plotmap(cortex_img,template_img,opacity=.5)

## Extracting a volume of interest at full resolution

We specify the rectangular volume of interest with two 3D points (the min and max point).

In [ ]:
minpt = (-3.979, -61.256, 3.906)
maxpt = (5.863, -55.356, -2.487)

A volume of interest is always defined in a particular space. Here, we define it in BigBrain space of course.

In [ ]:
voi = bigbrain.get_voi(minpt,maxpt)

Now we can extract a chunk from the BigBrain template a full resolution of 20 micron using this volume of interest specification. The extracted chunk sits nicely in BigBrain space. We can plot it in context, although it is extracted at a very different resolution.

In [ ]:
bigbrainchunk = bigbrain.get_template().fetch(resolution_mm=0.02,voi=voi)
plotting.view_img(bigbrainchunk,None)

We can reapply the volume of interest specification to extract chunks from other objects in this space as well, like parcellation maps. Here we use the cortex map of BigBrain, passing the specification to the fetch() method of the map. We can then plot it with the full-resolution chunk as template.

In [ ]:
cortexmap = siibra.parcellations['isocortex'].get_map("bigbrain")
mask = cortexmap.fetch(resolution_mm=0.2,voi=voi)
plotmap(mask,chunk,opacity=.2)